In [ ]:
# 1. Load dependencies and libraries
!pip install llama-index ragstack-ai pdfplumber transformers optimum[exporters] llama-index-llms-groq llama-index-embeddings-huggingface-optimum

import os
from getpass import getpass
import pdfplumber
from llama_index.core import Document, StorageContext, VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.vector_stores.astra_db import AstraDBVectorStore
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding
from llama_index.llms.groq import Groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 14.1 MB/s eta 0:00:00


In [ ]:
os.environ["ASTRA_DB_API_ENDPOINT"] = "https://7c7e7cda-bb99-491a-88a3-dd75d41c0b0d-us-east-2.apps.astra.datastax.com"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = "AstraCS:PEZqYNpmWDhAynggLjtTvqpM:ad3b93fe4f7c9609ec21668984073da866093171657e4611f4ccc88602235bcf"
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
llm = Groq(model="mixtral-8x7b-32768")
Settings.llm = llm


In [ ]:
# 2. Load a PDF and extract text
pdf_path = "/content/agenda.pdf"  # Replace with your actual PDF path

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

pdf_text = extract_text_from_pdf(pdf_path)

# New function to chunk text
def chunk_text(text, chunk_size=250):  # Reduced chunk size
    """Split text into smaller chunks of specified size."""
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# Chunk the extracted PDF text
text_chunks = chunk_text(pdf_text)

# Check if we have enough chunks
if len(text_chunks) < 10:
    print("Warning: Not enough chunks created. Consider reducing the chunk size further or checking the PDF content.")

print(f"Number of chunks created: {len(text_chunks)}")
for idx, chunk in enumerate(text_chunks):
    print(f"Chunk {idx + 1} ({len(chunk)} characters):\n{chunk}\n")

# Create Document objects for each chunk
documents = [Document(text=chunk) for chunk in text_chunks]

Number of chunks created: 36
Chunk 1 (250 characters):
Agenda
Global IndiaAI Summit 2024 3-4 July 2024, New Delhi
Day 1 - Wednesday, July 3
Start Session Description
10:00 - 11:00
Opening Ceremony (Auditorium 1)
11:00 – 13:30
GPAI Session (@ Summit Room) : Executive Council Meeting
(By Invitation only)
1

Chunk 2 (250 characters):
1:30 - 13:00 Side Event 1 (@ The session is about learning and sharing knowledge on Large
Auditorium 1): Language Models (LLMs) and Large Multimodal Models (LMMs),
with the aim to understand the unique linguistic and cultural
IndiaAI: Large diversity

Chunk 3 (250 characters):
 inherent to India, examining how LLMs can effectively
Language Model address challenges associated with multilingualism. Furthermore,
the session will cover the ethical considerations and biases linked
to these models, promoting a discussion on resp

Chunk 4 (250 characters):
onsible AI
practices, including fairness, inclusiveness, misinformation
mitigation, and intellectual property r

In [ ]:

# 3. Set up the embedding model
OptimumEmbedding.create_and_save_optimum_model(
    "BAAI/bge-small-en-v1.5", "./bge_onnx"
)

embed_model = OptimumEmbedding(folder_name="./bge_onnx")

# Check embedding dimension
test_embedding = embed_model.get_text_embedding("Test sentence")
print(f"Embedding dimension: {len(test_embedding)}")

Settings.embed_model = embed_model

Framework not specified. Using pt to export the model.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: BertModel *****
Using framework PyTorch: 2.2.1+cu121
Overriding 1 configuration item(s)
	- use_cache -> False


Saved optimum model to ./bge_onnx. Use it with `embed_model = OptimumEmbedding(folder_name='./bge_onnx')`.
Embedding dimension: 384


In [ ]:

# 4. Send data to vector store and store as embedding
# Generate a unique collection name
collection_name = f"pdf_embeddings_{uuid.uuid4().hex[:8]}"

astra_db_store = AstraDBVectorStore(
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
    collection_name=collection_name,
    embedding_dimension=len(test_embedding),
)

storage_context = StorageContext.from_defaults(vector_store=astra_db_store)

# Store all documents in the vector store
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, show_progress=True)

Parsing nodes:   0%|          | 0/36 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/36 [00:00<?, ?it/s]

In [ ]:
# 5. Query the data
query_engine = index.as_query_engine()

# Example query
query_string_1 = "summary of event?"
response = query_engine.query(query_string_1)
print(query_string_1)
print(response.response)

# Retrieve nodes with scores
retriever = index.as_retriever(
    vector_store_query_mode="default",
    similarity_top_k=10,
)

nodes_with_scores = retriever.retrieve(query_string_1)
print(f"\nFound {len(nodes_with_scores)} nodes.")
for idx, node_with_score in enumerate(nodes_with_scores):
    print(f" [{idx}] score = {node_with_score.score:.4f}")
    print(f" text = {node_with_score.node.text[:180]} ...")

summary of event?
The context describes three side events taking place at a summit. The first event, from 1:30 to 13:00 in Auditorium 1, is focused on Large Language Models (LLMs) and Large Multimodal Models (LMMs), with the goal of understanding their unique linguistic and cultural diversity. The second event, a Ministerial Council Meeting, is scheduled for the same time frame but is being held in the Summit Room and is by invitation only. The third event, from 14:30 to 16:00 in Auditorium 1, consists of case study presentations and panel discussions, bringing together technical experts to share their experiences in real-world settings.

Found 10 nodes.
 [0] score = 0.8583
 text = 1:30 - 13:00 Side Event 1 (@ The session is about learning and sharing knowledge on Large
Auditorium 1): Language Models (LLMs) and Large Multimodal Models (LMMs),
with the aim to  ...
 [1] score = 0.8538
 text = on (@ Summit Room) : Ministerial Council Meeting
(By Invitation only)
14.30 -16:00 Side Event 3 